In [ ]:
# Load data

import pandas as pd

data = pd.read_csv("../data/Boiler_emulator_dataset_new.csv")
data

In [ ]:
# Extract classes
classes = data["Class"].unique()
classes

In [ ]:
# Explore condition (Not part of input features)

conditions = {}
for c in classes:
    conditions[c] = data[data["Class"] == c]["Condition"].unique()

conditions